# Introduction to Code Contributions
The three contributions presented in the last chapter are implemented in different modules of the python code. To distinguish between features of one time step and aggregated features, the aggregated features are referred with ```X```, features of one time step with ```X_t``` and features of an particular individual with ```X_i```.

In [ ]:
import sys
import numpy as np
sys.path.append("../")

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from aif360.algorithms.inprocessing import PrejudiceRemover

from long_term_fairness.ltf_plot import LongTermFairnessPlot
from long_term_fairness.ltf_data.group_data_generator import DataGenerator as GrpDataGen
from long_term_fairness.ltf_aif.longterm_aif import AifLongTermMetric, AifLongTermPrediction

# 1. Data Generator
The data generator implements the dynamic data generation. The details about the data generation are explained in the previous section.

In [ ]:
# Initialization of the generator. The provided values are the default values.
generator = IndDataGen(mean_pos=[10, 7],             # The mean for the positive cluster.
                       cov_pos=[[1.2, 0], [0, 1.3]], # Covariance for the positive cluster.
                       mean_neg=[0, 1],              # The mean for the negative cluster.
                       cov_neg=[[1, 0], [0, 1.2]],   # Covariance for the negative cluster.
                       degree=3,                     # Number of previous times steps t considered in dynamics.
                       discrimination_factor=.7,     # The fraction of positive and negative labels in the first step.
                       num_positive_label=500,       # Number of positive instances.
                       num_negative_label=500,       # Number of negative instances.
                       neg_label=0, # The label considered negative.
                       pos_label=1, # The label considered positive.
                       neg_class=0, # The protected attribute class considered negative.
                       pos_class=1) # The protected attribute class considered positive.

In [ ]:
# The generator takes three arguments, the features X, true labels _y, and predictions y_hat.
#generator.sample(X,      # 3D all previous features of shape [t, n, m]
#                 _y, 
#                 y_hat


In [ ]:
# When called with None, None, None the generator samples the initial data as explained in the previous section.
X, x_sens, y = generator.sample(None, None, None)

In [ ]:
# Otherwise
X_new, x_sense_new, y_new = genertor.sample(X, x_sens, y)

In [ ]:
generator.get_label()

# 2. AIF360 Wrappers
Two wrapper classes provide access to aif360 metrics and functions to be used in the long term framework.

## 2.1. AIF360 Long Term Prediction
Interface to use AIF360s algorithms for the long term plot

In [ ]:
clf = AifLongTermPrediction(clf=PrejudiceRemover(), 
                            pos_class=1, 
                            neg_class=0, 
                            pos_label=1, 
                            neg_label=0)

In [ ]:
clf.fit(X, 
        X_sens, 
        y)

In [ ]:
clf.predict(X, 
            X_sens)

## 2.1. AIF360 Long Term Metric
The class provides an interface to use AIF360 metrics for long term prediction.

In [ ]:
# The default metrics are accuracy and disparate impact. It is possible to provide more than two metrics
# but the plot is unclear then. Any classification metric from aif 360 can be passed as string.
# https://aif360.readthedocs.io/en/latest/modules/metrics.html#classification-metric
metric = AifLongTermMetric(metrics=["accuracy", "disparate_impact"],
                           pos_class=1, 
                           neg_class=0, 
                           pos_label=1, 
                           neg_label=0)

In [ ]:
metric.metric(X_t, 
              X_sens_t, 
              y_t, 
              y_hat_t)

# 3. Long Term Plot
Runs decision process over several generations.

In [ ]:
l = LongTermFairnessPlot(generator,     # must implement .sample() function
                         clf,           # must implement fit(X, X_sens, y) and predict(X, X_sense)
                         metric.metric, # function
                         x_lim=[-3, 12], 
                         y_lim=[-1, 13])